# Module 12.2: NumPy

Recall that we always import NumPy using the standard `np` abbreviation.

In [1]:
import numpy as np

## Broadcasting in NumPy

Broadcasting is a more advanced concept in numpy. It describes how numpy treats arrays with different shapes during arithmetic operations.

Under certain constraints, the smaller array is “broadcast” across the larger array to have a compatible shape (very efficient). There are also cases where broadcasting is a bad idea, but we will not cover this part.

In [6]:
# element-wise numpy operation
a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 2.0, 2.0])
a * b

array([2., 4., 6.])

In [7]:
a = np.array([1.0, 2.0, 3.0])
b = 2.0
a * b

array([2., 4., 6.])

In the simplest example of broadcasting below, the scalar `b` is stretched to become an array of with the same shape as `a` so the shapes are compatible for element-wise multiplication.

<div>
<img src="attachment:image.png" width="500"/>
</div>

sourse: https://numpy.org/doc/1.20/user/theory.broadcasting.html

The Broadcasting rule: **the size of the trailing axes for both arrays in an operation must either be the same size or one of them must be one.** (trailing axes -- rightmost dimension)

**Example 1**: 

Numpy array A (a color image) has shape (256, 256, 3), the size of its rightmost dimension is 3.

Numpy array B (a scalar) has shape (3,), the size of its rightmost dimension is also 3.

So B can be broadcasted, it will be streched to (1, 1, 3) first (3 will still represent the size of the rightmost dimension), and then be streched to (256, 256, 3). This is its final shape before the element-wise operations.

**Example 2**: 

Numpy array A (4-dimension) has shape (8, 1, 6, 1), the size of its rightmost dimension is 1.

Numpy array B (3-dimension) has shape (7, 1, 5), the size of its rightmost dimension is 5.

So B will be broadcasted, it will be streched to (1, 7, 1, 5) first (5 will still represent the size of the rightmost dimension), and then be streched to (8, 7, 6, 5). This is its final shape before the element-wise operations.

And A will also be broadcasted, it will be streched from (8, 1, 6, 1) to (8, 7, 6, 5).

**Example 3**: 

<div>
<img src="attachment:image.png" width="500"/>
</div>

sourse: https://numpy.org/doc/1.20/user/theory.broadcasting.html

**Example 4**: 


<div>
<img src="attachment:image.png" width="500"/>
</div>

sourse: https://numpy.org/doc/1.20/user/theory.broadcasting.html

## Timing a computation with a NumPy array vs with a list

The main goal in this section is to see examples where code written using NumPy can run much faster than code written using base Python.

We'll make some examples using random numbers, so we start out with a **random number generator object**.

In [11]:
rng = np.random.default_rng()
type(rng)

numpy.random._generator.Generator

Now we make an array which has one million random integers from 1 to 10 (not including 10). We will be dividing by these numbers, so that's why we don't start with 0.  The `integers` method of `rng` is a NumPy array.

In [13]:
arr = rng.integers(1,10,size=10**6)
len(arr)

1000000

Here is the list version of `arr`.

In [14]:
mylist = list(arr)

Here are the first 5 elements in `arr`.

In [15]:
arr[:5]

array([3, 8, 5, 8, 8], dtype=int64)

Let's compute `1.5` divided by each of these five numbers.

In [16]:
1.5/arr[:5]

array([0.5   , 0.1875, 0.3   , 0.1875, 0.1875])

If we try to do the same thing with a list, we get an error, saying we can't combine the float 1.5 with the list `mylist` using the operator `/`.

In [17]:
1.5/mylist[:5]

TypeError: unsupported operand type(s) for /: 'float' and 'list'

Here is one way to make the list version, using a for loop.

In [18]:
newlist = []
for x in mylist[:5]:
    newlist.append(1.5/x)

In [19]:
newlist

[0.5, 0.1875, 0.3, 0.1875, 0.1875]

We haven't carefully studied list comprehension yet, but just for contrast, here is how we can make this same list using list comprehension. The list comprehension is definitely more elegant (often said to be more "Pythonic") than the for loop version.

In [20]:
[1.5/x for x in mylist[:5]]

[0.5, 0.1875, 0.3, 0.1875, 0.1875]

Let's see how the timings compare.

In [21]:
%%timeit
1.5/arr

2.31 ms ± 47.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit
newlist = []
for x in mylist:
    newlist.append(1.5/x)

1.61 s ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
[1.5/x for x in mylist]

1.62 s ± 34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Notice how the NumPy version is much faster than the list versions.  Two important reasons are: 1. the NumPy version is much faster.  One reason is that the NumPy version of division is *vectorized*, meaning all the divisions are performed at once (no looping is required); 2. the entries in `arr` are attached to a specific data type, so no time is spent checking the data types of the elements in `arr`.

Also notice that, even though the list comprehension version is more elegant than the for loop version, the timings are very similar.

## Counting using NumPy

We've seen examples of making arrays using NumPy and of performing basic arithmetic with NumPy arrays.  In this section, we'll see some examples of counting using NumPy.  For example, if we want to make a probability estimate using the formula "number of successes divided by number of experiments", then being able to count efficiently will be very helpful.

Our example will involve random numbers.  We'll start with a small number (`n = 20`) of random numbers, and then later we will increase `n`.  In general, you should imagine that the bigger the sample, the more accurate our probability estimate should be.

In [24]:
rng = np.random.default_rng()

In [25]:
n = 20

* Make a NumPy array of random integers between -50 and 50 (inclusive).  Also make a list with the same numbers.

In [26]:
arr = rng.integers(-50, 51, size=n)
print(arr)

[-12 -20  39   3 -46  49  46 -13  16 -42   7 -14  -6 -43 -26  38 -25 -28
  28  -5]


Here is the list version of the array `arr`.  (Be sure to always use a somewhat random variable name like `mylist`, as opposed to something like `list`, because `list` is already a built-in term in Python.)

In [27]:
mylist = list(arr)

* How many of the integers are strictly less than 10?

Here is a for-loop approach to the above question.

In [28]:
count = 0
for x in mylist:
    if x < 10:
        count += 1

In [29]:
count

14

Here is the **list comprehension** version of making this count; again, notice how much more concise this is than the for-loop approach.

In [30]:
len([x for x in mylist if x < 10])

14

To explain the NumPy approach to counting, it helps to actually look at the values in `arr`.

In [31]:
arr

array([-12, -20,  39,   3, -46,  49,  46, -13,  16, -42,   7, -14,  -6,
       -43, -26,  38, -25, -28,  28,  -5], dtype=int64)

The expression `arr < 10` creates what is called a Boolean array (meaning an array of `True`s and `False`s).  One way to think about this is, it is broadcasting the `< 10` portion to all of the entries in `arr`.  Be sure you understand how the following Boolean array relates to the numbers in `arr`.

In [32]:
arr < 10

array([ True,  True, False,  True,  True, False, False,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True])

There are many different possible ways to count `True` in this Boolean array.  Usually we prefer using NumPy's `count_nonzero` function.  

Remember that `True` corresponds to `1` and that `False` corresponds to `0`, so in a Boolean array, counting nonzero elements is the same as counting how often `True` occurs.

In [33]:
np.count_nonzero(arr < 10)

14

* At what indices do these integers less than 10 occur?

We use the NumPy function `np.nonzero` to find indices where the value is `True` (compare this to `np.count_nonzero` which we found above).  Notice that the resulting output is *not* a NumPy array... the output is a length-1 tuple which contains a NumPy array.

In [34]:
np.nonzero(arr < 10)

(array([ 0,  1,  3,  4,  7,  9, 10, 11, 12, 13, 14, 16, 17, 19],
       dtype=int64),)

In [35]:
type(np.nonzero(arr < 10))

tuple

* Make a new array containing only these integers.

We first show the for-loop approach.

In [36]:
newlist = []
for x in mylist:
    if x < 10:
        newlist.append(x)

In [37]:
newlist

[-12, -20, 3, -46, -13, -42, 7, -14, -6, -43, -26, -25, -28, -5]

Here is the list comprehension aproach.  We basically already did this approach above (the only difference is that we included a `len` wrapper above).

In [38]:
[x for x in mylist if x < 10]

[-12, -20, 3, -46, -13, -42, 7, -14, -6, -43, -26, -25, -28, -5]

In [39]:
arr

array([-12, -20,  39,   3, -46,  49,  46, -13,  16, -42,   7, -14,  -6,
       -43, -26,  38, -25, -28,  28,  -5], dtype=int64)

In [40]:
arr < 10

array([ True,  True, False,  True,  True, False, False,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True])

The following is what is called "Boolean indexing" or "Applying a Boolean mask".  You should read `arr[arr < 10]` similarly to how you would read `arr[:5]`.  Both are indicating which elements in `arr` to keep.  In the case of `arr[arr < 10]`, the part inside the brackets is a Boolean array, and we keep all the elements where `True` occurs.

In [41]:
arr[arr < 10]

array([-12, -20,   3, -46, -13, -42,   7, -14,  -6, -43, -26, -25, -28,
        -5], dtype=int64)

* If you pick a random integer from such an array, what's the probability it is less than 10?

To get an accurate probability estimate, we want to use as large a value of `n` as can be handled in a reasonably short amount of time; ten million works well in this case.

In [42]:
n = 10**7

In [43]:
arr = rng.integers(-50, 51, size=n)
mylist = list(arr)

Here is the for-loop approach.  We think of `count` as the number of successes and we think of `n` as the number of experiments.

In [44]:
count = 0
for x in mylist:
    if x < 10:
        count += 1
count/n

0.59369

In [45]:
%%timeit
count = 0
for x in mylist:
    if x < 10:
        count += 1
count/n

2.47 s ± 62.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The list comprehension takes a similar amount of time (it's a little faster than the for-loop version).

In [46]:
%%timeit
len([x for x in mylist if x < 10])

2.35 s ± 39.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Again, the NumPy version is much faster, over 100 times faster than the list comprehension version (and nearly 200 times faster than the for-loop version).

In [47]:
%%timeit
np.count_nonzero(arr < 10)

12 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


To get a sense for how accurate our probability estimate is, we can run the estimate again with new random numbers and see how similar the probability is.

In [48]:
arr = rng.integers(-50, 51, size=n)

In both cases, we get a probability estimate of approximately 0.594.

In [49]:
np.count_nonzero(arr < 10)/n

0.5941024

There are 101 numbers from -50 to 50 (inclusive), and 60 of those are strictly less than 10, so the exact probability is 60/101.  That exact value, 60/101, is very close to our probability estimates, so `n = 10**7` in this case was a big enough sample size to get a very accurate probability estimate.

In [50]:
60/101

0.594059405940594

## Logic in Python and NumPy

General rule of thumb:
* In base Python, use `and`, `or`, `not`
* In NumPy (or pandas), use `&`, `|`, `~` (tilde)

In [51]:
rng = np.random.default_rng()
n = 20

arr = rng.integers(-50, 51, size=n)
mylist = list(arr)

In [52]:
mylist

[-4,
 -26,
 -30,
 -34,
 -45,
 -36,
 17,
 9,
 -16,
 -24,
 -10,
 4,
 -37,
 -34,
 -44,
 -46,
 -37,
 -47,
 1,
 -3]

* Find all the entries in `arr` and in `mylist` that are strictly between -10 and 10.

Here is the list comprehension solution.  We use `and` because we are doing this in base Python.

In [53]:
[x for x in mylist if (x > -10) and (x < 10)]

[-4, 9, 4, 1, -3]

Here is a more complicated approach, that's just here as an excuse to use `not` and `or`.

In [54]:
[x for x in mylist if not ((x <= -10) or (x >= 10))]

[-4, 9, 4, 1, -3]

Here is the NumPy version.

In [55]:
arr

array([ -4, -26, -30, -34, -45, -36,  17,   9, -16, -24, -10,   4, -37,
       -34, -44, -46, -37, -47,   1,  -3], dtype=int64)

In [56]:
arr > -10

array([ True, False, False, False, False, False,  True,  True, False,
       False, False,  True, False, False, False, False, False, False,
        True,  True])

In [57]:
arr < 10

array([ True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

If we try to use `and` spelled out between these two Boolean arrays, we will get an error.

In [58]:
(arr > -10) and (arr < 10)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

We need to use the `&` to combine these Boolean arrays.

In [59]:
(arr > -10) & (arr < 10)

array([ True, False, False, False, False, False, False,  True, False,
       False, False,  True, False, False, False, False, False, False,
        True,  True])

Here we find the values using Boolean indexing.

In [60]:
arr[(arr > -10) & (arr < 10)]

array([-4,  9,  4,  1, -3], dtype=int64)

Just for fun, here are some other ways to get the same arrays.  We use `~` to negate, and we use the vertical line `|` (sometimes called "pipe") for `or`.

In [61]:
arr[(~(arr <= -10)) & (~(arr >= 10))]

array([-4,  9,  4,  1, -3], dtype=int64)

The following is a more complicated example.  Notice how we replace `not` and `or` in the list version with `~` and `|` in this NumPy version.

In [62]:
arr[~((arr <= -10) | (arr >= 10))]

array([-4,  9,  4,  1, -3], dtype=int64)

## The `axis` keyword argument

* If you roll four distinct 6-sided dice, what is the probability that the biggest value is 5?

In [63]:
rng = np.random.default_rng()

Here is an example of simulating this experiment using a NumPy random number generator.

In [64]:
rng.integers(1, 7, size=4)

array([4, 4, 1, 3], dtype=int64)

Let's repeat this experiment 10 times using a for loop.  (If we want to repeat the experiment many times, then we may want to avoid the for loop.)  Think of `s` as standing for the number of successes.  In the following, we use `np.max` instead of the built-in Python function `max`.  In general, if you have a choice between using a NumPy version or a built-in Python version, you should always prefer the NumPy version.

In [65]:
exps = 10
s = 0

for i in range(exps):
    if np.max(rng.integers(1, 7, size=4)) == 5:
        s += 1

s/exps

0.4

What we just did is very slow.  Even for just one million experiments, the following took 13 seconds.  (It is slow enough that I wanted to use `%%time`, which runs once, instead of `%%timeit`, which runs multiple times and takes an average.)

In [66]:
%%time

exps = 10**6
s = 0

for i in range(exps):
    if np.max(rng.integers(1, 7, size=4)) == 5:
        s += 1

s/exps

Wall time: 14.7 s


0.284888

The key idea to making this computation more efficient is to work with a two-dimensional array, and to think of each row as corresponding to one experiment.

In [67]:
exps = 10
rng.integers(1, 7, size=(exps,4))

array([[2, 6, 5, 4],
       [4, 4, 1, 2],
       [1, 3, 4, 6],
       [4, 3, 1, 3],
       [1, 6, 3, 1],
       [2, 3, 5, 4],
       [6, 2, 5, 5],
       [4, 6, 4, 5],
       [3, 5, 2, 3],
       [5, 2, 1, 4]], dtype=int64)

We can't use `np.max` directly this context (at least, not without using the `axis` keyword argument), because `np.max` will find the maximum of all 40 numbers in this array.

In [68]:
exps = 10
arr = rng.integers(1, 7, size=(exps,4))
print(arr)
np.max(arr)

[[2 3 4 5]
 [5 2 6 4]
 [2 2 3 2]
 [4 5 5 3]
 [5 6 5 4]
 [1 4 2 4]
 [2 6 4 5]
 [1 4 1 2]
 [2 1 2 4]
 [6 3 3 6]]


6

As a first workaround, we could use NumPy's `apply_along_axis` function.  The documentation shows that the first three input arguments are: (1) a function, (2) an axis, and (3) an array.  In our case, the function will be `np.max` and the array will be `arr`.  The main topic of this video is, what is meant by the `axis` keyword argument?

In [8]:
help(np.apply_along_axis)

Help on function apply_along_axis in module numpy:

apply_along_axis(func1d, axis, arr, *args, **kwargs)
    Apply a function to 1-D slices along the given axis.
    
    Execute `func1d(a, *args, **kwargs)` where `func1d` operates on 1-D arrays
    and `a` is a 1-D slice of `arr` along `axis`.
    
    This is equivalent to (but faster than) the following use of `ndindex` and
    `s_`, which sets each of ``ii``, ``jj``, and ``kk`` to a tuple of indices::
    
        Ni, Nk = a.shape[:axis], a.shape[axis+1:]
        for ii in ndindex(Ni):
            for kk in ndindex(Nk):
                f = func1d(arr[ii + s_[:,] + kk])
                Nj = f.shape
                for jj in ndindex(Nj):
                    out[ii + jj + kk] = f[jj]
    
    Equivalently, eliminating the inner loop, this can be expressed as::
    
        Ni, Nk = a.shape[:axis], a.shape[axis+1:]
        for ii in ndindex(Ni):
            for kk in ndindex(Nk):
                out[ii + s_[...,] + kk] = func1d(arr[ii 

In [9]:
arr

array([[1, 4, 5, 1],
       [6, 5, 4, 4],
       [2, 1, 3, 3],
       [6, 6, 1, 6],
       [6, 6, 6, 6],
       [1, 4, 2, 2],
       [4, 1, 3, 3],
       [5, 1, 4, 5],
       [3, 5, 3, 5],
       [3, 4, 6, 1]])

Think of `axis=1` as plugging in one row at a time, and think of `axis=0` as plugging in one column at a time.  In the following example, we get 5 as our first value, because 5 is the maximum of the top row.  We get 6 as our next value, because 6 is the maximum of the next row, and so on.

In [10]:
np.apply_along_axis(np.max, axis=1, arr=arr)

array([5, 6, 3, 6, 6, 4, 4, 5, 5, 6])

What if we used `axis=0` instead?  Then we will get four 6s, because there are four columns, and each column has a 6 as its maximum.

In [11]:
np.apply_along_axis(np.max, axis=0, arr=arr)

array([6, 6, 6, 6])

Note that we can think of the Boolean array as containing 1s and 0s (even though it actually contains Trues and Falses), and if we take its average, that is the same as "number of Trues divided by total number".

So we make a Boolean array `np.apply_along_axis(np.max, axis=1, arr=arr) == 5`, and then we compute its average.

In [69]:
exps = 10**6

arr = rng.integers(1, 7, size=(exps,4))
(np.apply_along_axis(np.max, axis=1, arr=arr) == 5).mean()

0.284285

This method is faster than our for loop method from above, but it still takes about 5 seconds, which is still fairly slow for only having one million experiments. 

In [72]:
%%time

exps = 10**6

arr = rng.integers(1, 7, size=(exps,4))
(np.apply_along_axis(np.max, axis=1, arr=arr) == 5).mean()

Wall time: 6.04 s


0.285189

The problem above is that `np.apply_along_axis` is very general, and is not optimized to the specific maximum computation we're making.  We will replace it by using the `max` method of `arr`.  If we call `max()` on its own, we get the maximum of the whole array, which is not what we want.

In [17]:
exps = 10

arr = rng.integers(1, 7, size=(exps,4))
print(arr)
arr.max()

[[6 6 5 6]
 [5 1 5 5]
 [4 5 4 5]
 [1 3 2 3]
 [2 4 5 6]
 [1 1 1 5]
 [1 2 1 3]
 [2 5 6 1]
 [5 6 1 1]
 [2 1 2 6]]


6

If we check the documentation for `arr.max` (notice that you get documentation for this sort of method by calling `help(arr.max)`, not `help(arr.max()` and not `help(max)`), we see that this `max` method also accepts an `axis` keyword argument, just like `np.apply_along_axis`.

In [18]:
help(arr.max)

Help on built-in function max:

max(...) method of numpy.ndarray instance
    a.max(axis=None, out=None, keepdims=False, initial=<no value>, where=True)
    
    Return the maximum along a given axis.
    
    Refer to `numpy.amax` for full documentation.
    
    See Also
    --------
    numpy.amax : equivalent function



Here is an example of using the `axis` keyword argument on a 10-row, 4-column NumPy array.

In [73]:
exps = 10

arr = rng.integers(1, 7, size=(exps,4))
print(arr)
arr.max(axis=1)

[[2 6 4 3]
 [2 2 6 6]
 [2 6 3 4]
 [5 6 4 5]
 [6 2 5 3]
 [6 3 5 6]
 [4 4 4 4]
 [3 6 2 5]
 [1 6 4 2]
 [3 2 1 3]]


array([6, 6, 6, 6, 6, 6, 4, 6, 6, 3], dtype=int64)

In [74]:
exps = 10

arr = rng.integers(1, 7, size=(exps,4))
(arr.max(axis=1) == 5)

array([False, False, False,  True,  True,  True, False,  True,  True,
       False])

If we try to use `axis=1` as an input argument to `mean` in this case, we will get an error, because the array is one-dimensional, so `axis=1` does not exist.

In [21]:
exps = 10

arr = rng.integers(1, 7, size=(exps,4))
(arr.max(axis=1) == 5).mean(axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

We could do `axis=0`, but for a one-dimensional NumPy array, we might as well just not provide an `axis` argument.

In [22]:
exps = 10

arr = rng.integers(1, 7, size=(exps,4))
(arr.max(axis=1) == 5).mean(axis=0)

0.4

Here is the best version of our computation.

In [24]:
exps = 10**6

arr = rng.integers(1, 7, size=(exps,4))
(arr.max(axis=1) == 5).mean()

0.285484

The code is not only concise but it is also very efficient.  Whereas even our NumPy code above took over 5 seconds, this is about one hundred times faster.

In [25]:
%%time
exps = 10**6

arr = rng.integers(1, 7, size=(exps,4))
(arr.max(axis=1) == 5).mean()

CPU times: user 46.5 ms, sys: 5.92 ms, total: 52.4 ms
Wall time: 51.3 ms


0.284911

## An exact probability

We ask the same question as last time, but now we will compute the exact probability.

* If you roll four distinct 6-sided dice, what is the probability that the biggest value is 5?

Often computing the exact probability (at least using an exhaustive search where we check every possibility) is not feasible, but in this case, there are only $6^4$ possible values (six different values for each of the four dice), so it's no problem to compute all of the possibilities.

In [75]:
6**4

1296

Here is a correct but inelegant way to make a list of all possible quadruples of integers from 1 to 6 (including 6).

In [76]:
all_dice = []
for i in range(1,7):
    for j in range(1,7):
        for k in range(1,7):
            for l in range(1,7):
                all_dice.append((i,j,k,l))

As a reality check, this list does have the correct length.

In [77]:
len(all_dice)

1296

Here are the first ten values in `all_dice`.  Do you see why it begins this way, and not for example `[(1,1,1,1), (2,1,1,1), ...]`?

In [5]:
all_dice[:10]

[(1, 1, 1, 1),
 (1, 1, 1, 2),
 (1, 1, 1, 3),
 (1, 1, 1, 4),
 (1, 1, 1, 5),
 (1, 1, 1, 6),
 (1, 1, 2, 1),
 (1, 1, 2, 2),
 (1, 1, 2, 3),
 (1, 1, 2, 4)]

What we really want to do is compute the Cartesian product of `[1,2,3,4,5,6]` with itself four times.  The standard way to do that in Python is to use the `itertools` library, which is part of the standard Python libraries (meaning it does not need to be installed).  We won't import the whole module; we'll just import its `product` function.

In [6]:
from itertools import product

In [7]:
help(product)

Help on class product in module itertools:

class product(builtins.object)
 |  product(*iterables, repeat=1) --> product object
 |  
 |  Cartesian product of input iterables.  Equivalent to nested for-loops.
 |  
 |  For example, product(A, B) returns the same as:  ((x,y) for x in A for y in B).
 |  The leftmost iterators are in the outermost for-loop, so the output tuples
 |  cycle in a manner similar to an odometer (with the rightmost element changing
 |  on every iteration).
 |  
 |  To compute the product of an iterable with itself, specify the number
 |  of repetitions with the optional repeat keyword argument. For example,
 |  product(A, repeat=4) means the same as product(A, A, A, A).
 |  
 |  product('ab', range(3)) --> ('a',0) ('a',1) ('a',2) ('b',0) ('b',1) ('b',2)
 |  product((0,1), (0,1), (0,1)) --> (0,0,0) (0,0,1) (0,1,0) (0,1,1) (1,0,0) ...
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /

Here is an example of getting all possible (ordered) pairs of integers from 1 to 6.  It returns an itertools `product` object, but we can convert that object into a list if we want to see all the pairs at once.

In [8]:
product(range(1,7), repeat=2)

In [9]:
list(product(range(1,7), repeat=2))

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 6)]

Here is another example of using the `product` function without using the `repeat` keyword argument.  In this case, we're getting the Cartesian product of the list `["a", "b", "c"]` with the list `[5, 6]`.  In total, this product has $3 \cdot 2 = 6$ elements.

In [10]:
list(product(["a","b","c"], [5, 6]))

[('a', 5), ('a', 6), ('b', 5), ('b', 6), ('c', 5), ('c', 6)]

One more example.

In [12]:
list(product(["a","b","c"], repeat=3))

[('a', 'a', 'a'),
 ('a', 'a', 'b'),
 ('a', 'a', 'c'),
 ('a', 'b', 'a'),
 ('a', 'b', 'b'),
 ('a', 'b', 'c'),
 ('a', 'c', 'a'),
 ('a', 'c', 'b'),
 ('a', 'c', 'c'),
 ('b', 'a', 'a'),
 ('b', 'a', 'b'),
 ('b', 'a', 'c'),
 ('b', 'b', 'a'),
 ('b', 'b', 'b'),
 ('b', 'b', 'c'),
 ('b', 'c', 'a'),
 ('b', 'c', 'b'),
 ('b', 'c', 'c'),
 ('c', 'a', 'a'),
 ('c', 'a', 'b'),
 ('c', 'a', 'c'),
 ('c', 'b', 'a'),
 ('c', 'b', 'b'),
 ('c', 'b', 'c'),
 ('c', 'c', 'a'),
 ('c', 'c', 'b'),
 ('c', 'c', 'c')]

The above example, using `repeat`, is the exact same as this next example, which explicitly lists the list three times.

In [13]:
list(product(["a","b","c"], ["a","b","c"], ["a","b","c"]))

[('a', 'a', 'a'),
 ('a', 'a', 'b'),
 ('a', 'a', 'c'),
 ('a', 'b', 'a'),
 ('a', 'b', 'b'),
 ('a', 'b', 'c'),
 ('a', 'c', 'a'),
 ('a', 'c', 'b'),
 ('a', 'c', 'c'),
 ('b', 'a', 'a'),
 ('b', 'a', 'b'),
 ('b', 'a', 'c'),
 ('b', 'b', 'a'),
 ('b', 'b', 'b'),
 ('b', 'b', 'c'),
 ('b', 'c', 'a'),
 ('b', 'c', 'b'),
 ('b', 'c', 'c'),
 ('c', 'a', 'a'),
 ('c', 'a', 'b'),
 ('c', 'a', 'c'),
 ('c', 'b', 'a'),
 ('c', 'b', 'b'),
 ('c', 'b', 'c'),
 ('c', 'c', 'a'),
 ('c', 'c', 'b'),
 ('c', 'c', 'c')]

We're now ready to use `product` to simulate every possible roll of four distinct six-sided dice.

In [14]:
temp_list = list(product(range(1,7), repeat=4))

In [15]:
len(temp_list)

1296

Let's convert this to a NumPy array, so we can use some of the earlier methods from this week.

In [16]:
arr = np.array(temp_list)

The array `arr` has 1296 rows and 4 columns.  We think of each row as corresponding to one possible outcome.

In [17]:
arr.shape

(1296, 4)

We want to know, for what proportion of the rows, is the maximum value in that row equal to 5?  Because we want to get the maximum of each row, we use the `axis=1` keyword argument.  (If we wanted to get the maximum of each column, we would instead use `axis=0`.)  We compute the total number of "successes" by using `sum`, and then we divide by the number of rows, which is `arr.shape[0]`.

In [19]:
(arr.max(axis=1) == 5).sum()/arr.shape[0]

0.2847222222222222

An abbreviation is to apply the `mean` method to our Boolean array.

In [20]:
(arr.max(axis=1) == 5).mean()

0.2847222222222222

Just for fun, here is another computation of the exact probability.  The first number in the difference, $(5/6)^4$, is equal to the probability that none of the dice lands 6, and then we are subtracting the probability that none of the dice lands 5 or 6.

In [21]:
(5/6)**4-(4/6)**4

0.2847222222222223

## `any` and `all`

Make a 100 row, 4 column NumPy array `arr` of random real numbers between 0 and 1.

In [2]:
rng = np.random.default_rng()

In [4]:
arr = rng.random(size=(100,4))

* Find the sub-array of `arr` containing all rows in which at least one number is bigger than 0.9.

Here are the first 5 rows of `arr`.

In [6]:
arr[:5]

array([[0.02679582, 0.39964722, 0.38608334, 0.9557704 ],
       [0.62903247, 0.17981028, 0.48643802, 0.16059557],
       [0.49612872, 0.34270062, 0.00862007, 0.77873829],
       [0.55550408, 0.37485681, 0.20417908, 0.82782464],
       [0.51663663, 0.85479878, 0.48625127, 0.88475327]])

Here is a Boolean array corresponding to which elements are strictly greater than 5.

In [7]:
arr[:5] > 0.9

array([[False, False, False,  True],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])

For each row, we want to know if any value in that row is `True`.  To compute this, we will use the `any` method, together with `axis=1`.  The reason we're using `axis=1` is because we are plugging in a full row at a time.

In [8]:
(arr[:5] > 0.9).any(axis=1)

array([ True, False, False, False, False])

If we wanted to count how many of these first 5 rows contain a number bigger than 0.9, we could use `sum`.

In [9]:
(arr[:5] > 0.9).any(axis=1).sum()

1

The question asked for us to find the rows which contain a number bigger than 0.9, so we will use Boolean indexing.  We are plugging in the `array([ True, False, False, False, False])` from above into the square brackets, which is like saying, keep the top row, discard the next four rows.

In [10]:
arr[:5][(arr[:5] > 0.9).any(axis=1)]

array([[0.02679582, 0.39964722, 0.38608334, 0.9557704 ]])

We now do the same thing to the full array.  This will be a little easier to read, because we can get rid of the `[:5]` slices.

In [11]:
arr[(arr > 0.9).any(axis=1)]

array([[0.02679582, 0.39964722, 0.38608334, 0.9557704 ],
       [0.89128322, 0.97335335, 0.1821295 , 0.52423414],
       [0.4629412 , 0.71844233, 0.91362938, 0.04770196],
       [0.61030793, 0.15565872, 0.15616296, 0.9572168 ],
       [0.45523301, 0.85912337, 0.34660356, 0.91882733],
       [0.14055923, 0.86443333, 0.95413845, 0.94380163],
       [0.07850217, 0.96370185, 0.21801727, 0.22118187],
       [0.10730651, 0.56976149, 0.95367251, 0.7782287 ],
       [0.95705318, 0.11012904, 0.1380347 , 0.67057015],
       [0.95930433, 0.06501316, 0.67658178, 0.20159313],
       [0.2457492 , 0.67240127, 0.94358665, 0.90993476],
       [0.37975889, 0.92498652, 0.99327127, 0.7283782 ],
       [0.37093209, 0.45237422, 0.91160131, 0.96612184],
       [0.52254037, 0.19969189, 0.94795525, 0.15420271],
       [0.90339743, 0.59821255, 0.62045249, 0.54939576],
       [0.48895101, 0.52105202, 0.96297772, 0.45230437],
       [0.35246537, 0.32415082, 0.86599985, 0.98355888],
       [0.32443071, 0.94130683,

* Find the sub-array of `arr` containing all rows in which no numbers are between 0.4 and 0.6.

One way to think about this is, we want the rows in which every element in that row is less than 0.4 or bigger than 0.6.  (We allow a single row to have both smaller and bigger elements; all we care about is that it does not contain any values that start with 0.4 or 0.5.)  Since we want a condition to be true for all the numbers in the row, we use `all(axis=1)`.

In [12]:
((arr < 0.4) | (arr > 0.6)).all(axis=1)

array([ True, False, False, False, False, False, False, False, False,
       False, False,  True, False,  True,  True, False,  True,  True,
       False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False,  True, False,  True,  True,
       False, False, False, False, False,  True,  True,  True, False,
        True,  True, False,  True, False, False, False, False, False,
        True,  True,  True, False, False, False,  True, False, False,
        True, False,  True,  True, False, False, False, False, False,
        True, False, False, False, False, False, False,  True, False,
        True, False, False, False,  True,  True, False, False,  True,
       False])

As a reality check, our Boolean array here is length 100.

In [13]:
len(((arr < 0.4) | (arr > 0.6)).all(axis=1))

100

Here again we're using Boolean indexing.  We're keeping all the rows of `arr` which satisfy this condition.  As you scroll through this array, notice how all the values that appear are either less than 0.4 or bigger than 0.6.

In [14]:
arr[((arr < 0.4) | (arr > 0.6)).all(axis=1)]

array([[0.02679582, 0.39964722, 0.38608334, 0.9557704 ],
       [0.61030793, 0.15565872, 0.15616296, 0.9572168 ],
       [0.64463239, 0.62136197, 0.83285561, 0.69102624],
       [0.19806438, 0.28789025, 0.16912325, 0.71827727],
       [0.14055923, 0.86443333, 0.95413845, 0.94380163],
       [0.07850217, 0.96370185, 0.21801727, 0.22118187],
       [0.95705318, 0.11012904, 0.1380347 , 0.67057015],
       [0.6703015 , 0.81869015, 0.31666259, 0.67068789],
       [0.74644917, 0.06370154, 0.34252816, 0.06391344],
       [0.06099061, 0.08149432, 0.29021873, 0.85693158],
       [0.80698662, 0.31211537, 0.68117698, 0.78836747],
       [0.95930433, 0.06501316, 0.67658178, 0.20159313],
       [0.2457492 , 0.67240127, 0.94358665, 0.90993476],
       [0.37975889, 0.92498652, 0.99327127, 0.7283782 ],
       [0.35246537, 0.32415082, 0.86599985, 0.98355888],
       [0.32443071, 0.94130683, 0.86154932, 0.95131523],
       [0.75210216, 0.38678934, 0.19343926, 0.65172332],
       [0.75785393, 0.97896958,

## From binary to decimal, version 1

$$
\begin{pmatrix} 1 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 1 & 1 & 1 & 1 \end{pmatrix} \mapsto \begin{pmatrix} 10 \\ 4 \\ 15 \end{pmatrix}
$$

* Write a function `to_bin` which takes in an m-by-n NumPy array of 0s and 1s, and as output returns a length m NumPy array of the corresponding integers, where we think of each row as representing the binary digits of an integer.

For example, we think of the top row as representing $1 \cdot 2^3 + 0 \cdot 2^2 + 1 \cdot 2^1 + 0 \cdot 2^0 = 10$.

Let's start by getting a random array that can be our test input.

In [78]:
rng = np.random.default_rng()

In [79]:
arr = rng.integers(2, size=(10,4))

We assign `m` to be the number of rows and `n` to be the number of columns, using what is called tuple unpacking.

In [80]:
m,n = arr.shape

In [81]:
m

10

In [82]:
n

4

We want to multiply the left-most column by $2^3$, the next column by $2^2$, and so on.  Here are the exponents.  The middle `-1` is telling NumPy where to stop (stop before -1), and the right-most `-1` is telling NumPy what the step size should be (go down by 1 each step).

In [7]:
np.arange(n-1, -1, -1)

array([3, 2, 1, 0])

Here are the coefficients we want to multiply the columns by.

In [8]:
2**np.arange(n-1, -1, -1)

array([8, 4, 2, 1])

Using broadcasting, it's very easy to make the multiplication.  And to add up along the rows, we use `sum(axis=1)`.

In [12]:
(arr*(2**np.arange(n-1, -1, -1))).sum(axis=1)

array([12,  7, 11, 15,  4,  1,  0, 12, 14,  9])

In [13]:
def to_bin(arr):
    _,n = arr.shape
    return (arr*(2**np.arange(n-1, -1, -1))).sum(axis=1)

In [14]:
to_bin(arr)

array([12,  7, 11, 15,  4,  1,  0, 12, 14,  9])

Here we check on a smaller array.

In [15]:
to_bin(np.array([[1,0],[1,1]]))

array([2, 3])

## From binary to decimal, version 2

$$
\begin{pmatrix} 1 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 1 & 1 & 1 & 1 \end{pmatrix} \mapsto \begin{pmatrix} 10 \\ 4 \\ 15 \end{pmatrix}
$$

* Write a new function, `to_bin2`, which takes in an m-by-n NumPy array of 0s and 1s, and as output returns a length m NumPy array of the corresponding integers, where we think of each row as representing the binary digits of an integer.

In [2]:
rng = np.random.default_rng()
arr = rng.integers(2, size=(10,4))

In [3]:
arr

array([[0, 1, 1, 1],
       [0, 0, 1, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 1],
       [0, 1, 1, 1],
       [1, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 0]])

Here is the "usual" way of converting from a string to an integer.

In [4]:
int("0111")

111

But we can also tell Python that the string should be thought of as being written in binary.  For example, 0111 in binary should be thought of as $4+2+1 = 7$.  In the following, we're specifying that the number is being written in "base-2".

In [5]:
int("0111", 2)

7

Our "usual" number system is written in base-10.

In [6]:
int("0111", 10)

111

Here we make a length-4 NumPy array.

In [7]:
z = np.array([0,1,1,1])

Or it's probably more elegant to just take the top row of `arr`.

In [8]:
z = arr[0]

In [9]:
z

array([0, 1, 1, 1])

How can we convert that into the string "0111"? In the following, we use list comprehension to put the integers 0, 1, 1, 1 into a list.

In [10]:
[x for x in z]

[0, 1, 1, 1]

Here we use list comprehension to put the strings "0", "1", "1", "1" into a list.  It is the same as the previous command, except that we convert each integer to a string using `str`.

In [11]:
[str(x) for x in z]

['0', '1', '1', '1']

Although `sum` function seems to be the same as concatenation (`+`), but they are different and it doesn't work here.

In [12]:
sum([str(x) for x in z])

TypeError: unsupported operand type(s) for +: 'int' and 'str'

The following is the suggested way in Python to concatenate all the strings in a list together.

In [13]:
''.join([str(x) for x in z])

'0111'

We want to apply that idea to every row in `arr`.  Let's define a helper function to use.  Later we will see a way to write this sort of short function in a more concise way, using "lambda functions".  

**Caution**:  A common mistake -- use something like `print` instead of `return`.  That's very incorrect.  If you want your function to have an output, you need to use `return`.

In [14]:
def helper(z):
    return ''.join([str(x) for x in z])

Let's test out our `helper` function on the 3rd row of `arr`.

In [15]:
helper(arr[3])

'1011'

In [16]:
arr[3]

array([1, 0, 1, 1])

In [17]:
arr

array([[0, 1, 1, 1],
       [0, 0, 1, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 1],
       [0, 1, 1, 1],
       [1, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 0]])

Here is a version `helper2` that treats the row as if it corresponds to binary digits.  We could do this all in one line, but it is more readable to break it up into two steps.

In [18]:
def helper2(z):
    temp = ''.join([str(x) for x in z])
    return int(temp, 2)

In [19]:
helper2(arr[3])

11

Here we're using `apply_along_axis` again.  The function we want to apply is `helper2`, and we are applying along rows, so we use `axis=1`.

In [20]:
def to_bin2(A):
    return np.apply_along_axis(helper2, axis=1, arr=A)

This doesn't work because `A` is not defined.

In [21]:
to_bin2(A)

NameError: name 'A' is not defined

In [22]:
arr

array([[0, 1, 1, 1],
       [0, 0, 1, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 1],
       [0, 1, 1, 1],
       [1, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 0]])

Notice how, for example, the top row corresponds to 7, the next row corresponds to 2, and so on.

In [23]:
to_bin2(arr)

array([ 7,  2,  9, 11,  7,  9,  5,  2, 12,  2])

## Raising errors

* Edit the `to_bin` function so that it raises an error if the input is not a NumPy array and it also raises an error if an entry is not 0 or 1.

In [2]:
rng = np.random.default_rng()
arr = rng.integers(2, size=(10,4))

In [4]:
arr

array([[1, 1, 1, 1],
       [0, 0, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 0, 1],
       [1, 1, 0, 1],
       [1, 1, 1, 1],
       [1, 1, 0, 0],
       [0, 0, 1, 1]])

In [5]:
type(arr)

numpy.ndarray

Here is the usual way to check the `type` of a variable, using the function `isinstance`.  It doesn't literally check that the type is exactly a NumPy array, but what this checks is usually good enough.  (Technically, it checks if `arr` is a NumPy array or is a subclass of the class NumPy array.)

In [6]:
isinstance(arr, np.ndarray)

True

In [7]:
isinstance(arr, list)

False

Here is how we raise an error in Python.  In this case, we will raise a TypeError.  Don't worry about learning the many different error types in Python; for Math 9, the important thing is to know the general syntax of how to raise an error.

In [8]:
def to_bin(arr):
    if not isinstance(arr, np.ndarray):
        raise TypeError("Input should be a NumPy array")
    _,n = arr.shape
    return (arr*(2**np.arange(n-1, -1, -1))).sum(axis=1)

In [9]:
to_bin(arr)

array([15,  3, 11, 14,  3,  5, 13, 15, 12,  3])

Notice how the error message we wrote, "Input should be a NumPy array", shows up in the following error message.

In [10]:
to_bin(list(arr))

TypeError: Input should be a NumPy array

Now we will work on the second portion, namely, checking that all of the entries are 0 or 1.

In [11]:
arr == 0

array([[False, False, False, False],
       [ True,  True, False, False],
       [False,  True, False, False],
       [False, False, False,  True],
       [ True,  True, False, False],
       [ True, False,  True, False],
       [False, False,  True, False],
       [False, False, False, False],
       [False, False,  True,  True],
       [ True,  True, False, False]])

In [12]:
arr == 1

array([[ True,  True,  True,  True],
       [False, False,  True,  True],
       [ True, False,  True,  True],
       [ True,  True,  True, False],
       [False, False,  True,  True],
       [False,  True, False,  True],
       [ True,  True, False,  True],
       [ True,  True,  True,  True],
       [ True,  True, False, False],
       [False, False,  True,  True]])

The array is acceptable if each entry is zero or one, so we use the vertical line `|` for "or".

In [13]:
(arr == 0) | (arr == 1)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

Because we are checking each entry (not each entry in a given row), we use the `all` method with no `axis` argument.

In [14]:
((arr == 0) | (arr == 1)).all()

True

In [15]:
def to_bin(arr):
    if not isinstance(arr, np.ndarray):
        raise TypeError("Input should be a NumPy array")
    if not ((arr == 0) | (arr == 1)).all():
        raise ValueError("All entries should be 0 or 1")
    _,n = arr.shape
    return (arr*(2**np.arange(n-1, -1, -1))).sum(axis=1)

In [16]:
to_bin(arr)

array([15,  3, 11, 14,  3,  5, 13, 15, 12,  3])

In [17]:
arr2 = arr

We haven't emphasized the following type of assignment much, but here is how you set the 5th row, 2nd column to be 6.

In [18]:
arr2[5,2] = 6

In [19]:
arr2

array([[1, 1, 1, 1],
       [0, 0, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 6, 1],
       [1, 1, 0, 1],
       [1, 1, 1, 1],
       [1, 1, 0, 0],
       [0, 0, 1, 1]])

Here this raises the error we chose, because one of the entries is 6.

In [20]:
to_bin(arr2)

ValueError: All entries should be 0 or 1

A big "gotcha" in Python is that our assignment above also changed the original `arr` array.  (NumPy does this to save memory space.)

In [21]:
to_bin(arr)

ValueError: All entries should be 0 or 1

Notice how `arr` got changed, even though we never changed it ourselves, we only changed `arr2`.

In [22]:
arr

array([[1, 1, 1, 1],
       [0, 0, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 6, 1],
       [1, 1, 0, 1],
       [1, 1, 1, 1],
       [1, 1, 0, 0],
       [0, 0, 1, 1]])

If you want to make sure the original array doesn't get changed, use the `copy` method.

In [23]:
arr3 = arr.copy()

In [24]:
arr3

array([[1, 1, 1, 1],
       [0, 0, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 6, 1],
       [1, 1, 0, 1],
       [1, 1, 1, 1],
       [1, 1, 0, 0],
       [0, 0, 1, 1]])

Here we set all the even-indexed columns in row 6 to be equal to 99.

In [25]:
arr3[6,::2] = 99

In [26]:
arr3

array([[ 1,  1,  1,  1],
       [ 0,  0,  1,  1],
       [ 1,  0,  1,  1],
       [ 1,  1,  1,  0],
       [ 0,  0,  1,  1],
       [ 0,  1,  6,  1],
       [99,  1, 99,  1],
       [ 1,  1,  1,  1],
       [ 1,  1,  0,  0],
       [ 0,  0,  1,  1]])

Because we used the `copy` method when we created `arr3`, the original array `arr` did not get changed by our assignment.

In [27]:
arr

array([[1, 1, 1, 1],
       [0, 0, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 6, 1],
       [1, 1, 0, 1],
       [1, 1, 1, 1],
       [1, 1, 0, 0],
       [0, 0, 1, 1]])

## Other useful links

Christopher's youtube video playlist: https://www.youtube.com/watch?v=w3HxawiC-7s&list=PLHfGN68wSbbIuSS-Y1Y5zYDPN59OaggpA